# Dive

from Erik Baker's "Decolessons".

This is the exaple profile used in Erik's fortran program at the end of the "Decolessons" paper. The dive is a 20-minute dive to 90 metres using GF 30/75 and 4 gas mixes:

 - a. 13% O<sub>2</sub>, 50% He, 37% N<sub>2</sub>
 - b. 36% O<sub>2</sub>, 64% N<sub>2</sub>
 - c. 50/50 O<sub>2</sub>/N<sub>2</sub>
 - d. 80% O<sub>2</sub>, 20% N<sub>2</sub>

Because there is helium, it's ZH-L16B with "5-minute" first tissue compartment. Rq is 1 and "oxygen deco factor" is 1 as well, i.e. not used.

Profile:

1. Descent to 90 msw at the rate of 23 msw/min on gas a: 3.9 minutes.
2. 16.9 minutes at 90 msw on gas a: 20 min total runtime.
3. Ascent to 51 metres at -10 msw/min, 1-minute deco stop, repeat for 48 and 45 msw w/ 1-minute stop at each depth.
4. Ascent to 42, 39, 36, 33 msw as above, but w/ 2-minute stops.
5. Ascent to 30 msw, 1-minute stop.
* Gas switch to mix b.
6. Ascent to 27 then 24 msw w/ 2-minute stops at each depth.
7. Ascent to 21 msw.
* Gas switch to mix c.
8. 3-minute stop at 21 msw.
9. Ascent to 18 msw and 4-minute stop.
10. Ascent to 15 msw and 6-minute stop.
11. Ascent to 12 msw and 7-minute stop.
12. Ascent to 9 msw.
* Gas switch to mix d.
13. 10-minute stop at 9 msw.
14. Ascent to 6 msw and 15-minute stop.
15. Ascent ot 3 msw and 31-minute stop.
16. Surface.

Erik calculates max tissue loading (leading tissue compartment) relative to Buhlman's original M-value and ends up with 91.2% at the surface. Without gradient factors that increases to 99.2%.

In [2]:
import diyzhl

#
#
class eriks_dive( object ) :
    
    # ctor
    #
    def __init__( self, verbose = False ) :

        self._T = 0
        self._S = 1.0
        self._P = self._S
        self._RQ = 1.0
        self._GFHi = 0.75
        self._GFlo - 0.30
        self._TCs = { "h" : [], "n" : [] }

# starting Pt (same for all TCs)
        sp = diyzhl.palv( Pamb = self._P, Q = 0.79, RQ = self._RQ )

# use ZH-L16Cb (skip over 4-minute TC)
        for tc in diyzhl.ZHL16N.keys() :
            if tc == 1 : continue
            self._TCs["n"].append( { 
                "t" : diyzhl.ZHL16N[tc]["t"], 
                "a" : diyzhl.ZHL16N[tc]["a"]["C"],
                "b" : diyzhl.ZHL16N[tc]["b"],
                "P" : sp
            } )
#
# He is 0
#
            self.TCs["h"].append( 0 )
        
    # descent to 90 msw at 23 m/min on gas mix a
    #
    def descent() :

        t = 3.9
        qn = 0.37
        qh = 0.5
        p = 10.0
        
        for i in range( len( self._TCs ) ) :

            pn = diyzhl.schreiner( Pi = self._TCs["n"][i]["P"], 
                          Palv = diyzhl.palv( Pamb = self._P, Q = qn, RQ = self._RQ ), 
                          t = t, 
                          R = diyzhl.arr( d0 = self._P, dt = p, t = t, Q = qn ),
                          k = diyzhl.kay( Th = self._TCs["n"][i]["t"] ) )
            
            ph = diyzhl.schreiner( Pi = self._TCs["h"][i]["P"], 
                          Palv = diyzhl.palv( Pamb = self._P, Q = qn, RQ = self._RQ ), 
                          t = t, 
                          R = diyzhl.arr( d0 = self._P, dt = p, t = t, Q = qn ),
                          k = diyzhl.kay( Th = self._TCs["h"][i]["t"] ) )
!FIXME            
            self._TCs[i]["P"] = p
            self._TCs[i]["C"] = diyzhl.buhlmann( Pn = self._TCs[i]["P"],
                                         an = self._TCs[i]["a"],
                                         bn = self._TCs[i]["b"],
                                         gf = self._GFHi )

        self._P = newP
        self._T += t
    
        if self._verbose :
            sys.stdout.write( "* At time %f, P %f:\n" % (self._T, self._P,) )
            pprint.pprint( self._TCs )
